# API Tutorial 01 - Download data from the Open Energy Platform using the API

<!-- keep img below title and without align="left"  -->
<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75"/>


## Learnings

This tutorials will enable you to:

- [Download data](#download-data)
- [Download filtered data](#download-filtered-data)
- [Download metadata](#download-metadata)

## Requirements

- Python packages:
  - `requests`
  - jupyter notebook, e.g. `notebook` or `jupyterlab`
  - `shapely` (only for geometry data conversion)
- Skills:
  - use python with jupyter notebooks


## Setup


In [1]:
# install required packages with: pip install requests
# import required packages
import json

import requests as req

# TODO: explain schema/topic/sandbox
# TODO: explain randint

topic = "model_draft"
table = "tutorial_example_table"
table_api_url = f"https://openenergyplatform.org/api/v0/schema/{topic}/tables/{table}/"

## Download data

requires execution of [setup](#setup)


In [2]:
res = req.get(table_api_url + "rows/")
data = res.json()

# show results in notebook
print(json.dumps(data, indent=4))

[
    {
        "id": 1,
        "name": "unit1",
        "is_active": true,
        "capacity_mw": 1.2,
        "installation_datetime_utc": "2010-02-03T00:00:00",
        "location": "01010000008FC2F5285C8F4A40713D0AD7A3B02040"
    },
    {
        "id": 2,
        "name": "unit2",
        "is_active": false,
        "capacity_mw": 2.1,
        "installation_datetime_utc": "2010-01-08T00:00:00",
        "location": null
    },
    {
        "id": 3,
        "name": "unit3",
        "is_active": true,
        "capacity_mw": 100.0,
        "installation_datetime_utc": "2010-01-02T10:30:00",
        "location": "0101000000EC51B81E85AB4B405C8FC2F5285C1D40"
    }
]


## Notes on data types

The api returns json serializable data, which is only bool, numerical, or string.
In the example, the value of `installation_datetime_utc` is a string representation
of a datetime object, the value of `location` a
[WKB](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry)
string representation of a geometry object.

Depending on the use case, the data needs to be converted after the download


In [3]:
from shapely import wkb
from datetime import datetime

data_converted = []
for row in data:
    row_converted = row.copy()
    # convert datetime string to datetime object
    row_converted["installation_datetime_utc"] = datetime.strptime(
        row_converted["installation_datetime_utc"], "%Y-%m-%dT%H:%M:%S"
    )
    # convert wkb string to shapely geoemtry object
    row_converted["location"] = wkb.loads(row_converted["location"])
    data_converted.append(row_converted)

## Download filtered data

requires execution of [setup](#setup)


In [4]:
# Add (multiple) where filters to the url
filter = "where=is_active=true&where=capacity_mw>10"
res = req.get(table_api_url + f"rows/?{filter}")
data = res.json()

# show results in notebook
print(json.dumps(data, indent=4))

[
    {
        "id": 3,
        "name": "unit3",
        "is_active": true,
        "capacity_mw": 100.0,
        "installation_datetime_utc": "2010-01-02T10:30:00",
        "location": "0101000000EC51B81E85AB4B405C8FC2F5285C1D40"
    }
]


## Download metadata

requires execution of [setup](#setup)


In [5]:
res = req.get(table_api_url + "meta/")
metadata = res.json()

# show (partial) results in notebook
print(json.dumps(metadata, indent=4))

{
    "id": "test_table",
    "keywords": [
        "energy",
        "installations",
        "geo"
    ],
    "resources": [
        {
            "name": "test_table",
            "schema": {
                "foreignKeys": [],
                "fields": [
                    {
                        "name": "id",
                        "description": "numerical id of this data point",
                        "type": "integer"
                    },
                    {
                        "name": "name",
                        "description": "name of installation",
                        "type": "string"
                    },
                    {
                        "name": "is_active",
                        "description": "true/false if installation is active",
                        "type": "boolean"
                    },
                    {
                        "name": "capacity_mw",
                        "description": "installed capacity in MW",
       

## About this tutorial

:oep-logo-sirop:

- Author: Christian Winger
- Copyright: [Öko-Institut](https://www.oeko.de) (2024)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Attribution: Öko-Institut (2024): API Tutorial 01- Download data from the Open Energy Platform using the API.
- Last update :oep-auto-lastupdate:
